In [1]:
#Importing data, and convert to log ratio for each of the columns
##Useful indices to know: column 0 is sample_id, last 12 columns are not proteins, 2nd to last column is cell viability.
## row 84 is the control row
import pandas as pd
# Load the data from the CSV file
data_path = r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\preprocessing\data.csv'
df_raw = pd.read_csv(data_path)
#pull out control row:
df_control_row = df_raw[df_raw['pert_id']=='control'] #pulls out the control row

## the control row is not actually complete, so we need to fill in the missing values
# count number of nas in control row
num_nas_control = df_control_row.isna().sum().sum()

#print statement debugging:                
#print(f'Number of missing values in control row: {num_nas_control}')

#for rows that are missing, fill in with the average of the values in the column:
import numpy as np
for column in df_control_row.columns:
    if df_control_row[column].values==np.nan:
        column_mean = df_raw[column].mean()  # Calculate the mean of the column in df_raw
        df_control_row[column]=column_mean

#LEGACY: FIGURING OUT CONTROL ROW INDEX
#control_row_index = df_raw[df_raw['pert_id'] == 'control'].index[0] # gets the control row index



#converting to log ratio for each of the columns:
df_log_ratio = df_raw.copy()
df_log_ratio.iloc[:,1:-12]=np.log(df_log_ratio.iloc[:,1:-12].values / df_control_row.iloc[:,1:-12].values)  # Log ratio calculation

df_log_ratio.iloc[:,-2]=np.log(df_raw.iloc[:,-2]/100)
df_log_ratio.iloc[84,:]=df_control_row
df_log_ratio.iloc[84,1:-12+-2]=0  # Set values for control row to 0 because its a log of itself


C:\Users\abdul\AppData\Local\Temp\ipykernel_31500\3109657842.py:7: DtypeWarning: Columns (8546) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(data_path)


In [2]:
#pulling out proteins that are targeted by drugs:
df_targets=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\preprocessing\prots_info.csv')
incomplete_threshold=.05
targeted_prots=[]
# truth_vals= zip(df_targets['NaN_control']==False and df_targets['NaN_prop_all_samples']<incomplete_threshold)
conditions = (df_targets['NaN_control'] == False) & (df_targets['NaN_prop_all_samples'] < incomplete_threshold)&(df_targets['targeting_pert_ids'].notna())
idx_targets=df_targets.index[conditions]
#######################print(idx_targets)
targeted_prots.append(df_targets.iloc[idx_targets,0])
targeted_prots=targeted_prots[0].tolist()
#####################print(f'Targeted proteins: {targeted_prots}') 48 total

#pulling out top 100 most variable columns that are over 95% complete and also have the control row filled in:
# Identify proteins that don't have control values
no_control_prots = df_targets.loc[df_targets['NaN_control'], 'proteins'].tolist()

highly_incomplete_prots=df_targets.loc[df_targets['NaN_prop_all_samples']>incomplete_threshold,'proteins'].tolist()

#drop columns that don't have control values
#df_LR_inc_noctrl_drop

df_log_ratio_incomplete_and_control_dropped=df_log_ratio.drop(columns=no_control_prots, errors='ignore')
df_log_ratio_incomplete_and_control_dropped=df_log_ratio_incomplete_and_control_dropped.drop(columns=highly_incomplete_prots, errors='ignore')
sorted_stds=df_log_ratio_incomplete_and_control_dropped.iloc[:,1:-12].std(axis=0,skipna=True).sort_values(ascending=False)
top_100_prots = sorted_stds.head(100).index.tolist()

print('top_100:'+str(top_100_prots))
print('targeted: '+str(targeted_prots))
print(len(targeted_prots))

top_100:['Q9Y6Y8', 'Q9Y6Y0', 'P02771', 'Q96PX9', 'O94886', 'Q96S21', 'Q6DHV7', 'Q9GZU1', 'P19823', 'P01023', 'Q8IUE6', 'Q96JY6', 'Q06033', 'Q8WY22', 'P02774', 'P23142', 'P02768', 'P36955', 'O60783', 'P62070', 'Q9NPD3', 'Q01130', 'Q9NRS6', 'Q9Y5A9', 'P01024', 'Q96FQ6', 'Q8NG11', 'P04264', 'Q9BQB6', 'P02788', 'Q9GZT6', 'Q86SK9', 'P13645', 'Q9BXK5', 'Q96EB1', 'Q14512', 'P41134', 'P08697', 'Q7Z7M9', 'P49914', 'O75190', 'P02649', 'P46779', 'Q16760', 'Q99808', 'Q15582', 'P31431', 'Q96CX6', 'O00189', 'P56693', 'Q96EH3', 'O15235', 'Q86WX3', 'Q96J84', 'P48357', 'P13747', 'Q8WUD1', 'P04114', 'Q9NVH2', 'Q9HBH0', 'Q16610', 'P05546', 'Q9H4B7', 'Q15063', 'Q8IUC6', 'Q6P4Q7', 'O43572', 'P17900', 'Q96QE5', 'Q9Y3A4', 'A1L188', 'Q96QD8', 'Q9P0J0', 'O75170', 'Q9UQN3', 'P05452', 'P08236', 'Q969Q0', 'P58004', 'P56589', 'Q92997', 'P63096', 'Q7Z7F7', 'P62314', 'O14763', 'Q4VCS5', 'P49207', 'P06454', 'Q9H467', 'Q13214', 'Q7Z698', 'O00750', 'Q9UH17', 'Q5TAQ9', 'Q9H9H4', 'P02751', 'Q969L2', 'Q9NVN3', 'Q8WV24', '

In [3]:
#writing to file, with the top 100 most variable proteins that meet these criteria and also those that are targeted by drugs:
#with targeted proteins, some don't have a control value and some are relatively incomplete. For now, I will filter those out and go from there.
#REVISIT though
data_subset=df_log_ratio[targeted_prots+top_100_prots]
#adding on the last 12 columns from df_log_ratio
data_subset = pd.concat([data_subset, df_log_ratio.iloc[:, -12:]], axis=1)
data_subset.to_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\preprocessing\finalized_starting_prots.csv', index=False)


In [4]:
### Writing code to get the actual elasticnet model. (lord help me)
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\preprocessing\finalized_starting_prots.csv')

def model_training(data,alpha=1, l1_ratio=.5):
    prots=data.iloc[1:,:-12]  # Exclude the last 12 columns (cell viability and other non-protein data)
    viability_col = data.iloc[1:,-2]  # Assuming the second to last column is cell viability
    x_train,x_test,y_train,y_test=train_test_split(prots, viability_col,train_size=.7)
    #handling data for the missing values:
    x_train.fillna(0, inplace=True), x_test.fillna(0, inplace=True), y_train.fillna(0, inplace=True), y_test.fillna(0, inplace=True)
    elastic_model= ElasticNet(alpha=alpha, l1_ratio=l1_ratio)  # Adjust alpha and l1_ratio as needed
    elastic_model.fit(x_train,y_train)
    y_test_pred = elastic_model.predict(x_test)
    mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    return elastic_model, mse, r2,(x_train, x_test, y_train, y_test)  # Return the model and metrics along with the train/test data


broken_models=[]
data_splits=[]
for i in range(1, 100): #doing 100 training iterations:
    e_model,mse,r2,data_split=model_training(data, alpha=1, l1_ratio=0.5)  # Adjust alpha and l1_ratio as needed
    if mse>1:
        broken_models.append(e_model)
        data_splits.append(data_split)
    print(f"Iteration {i}: "+f"Mean Squared Error: {mse}, R^2: {r2}")



Iteration 1: Mean Squared Error: 0.02225717404669669, R^2: -0.5805376838354237
Iteration 2: Mean Squared Error: 0.0900354378008645, R^2: 0.15516390437626904
Iteration 3: Mean Squared Error: 0.06288358945769791, R^2: 0.00802211104307704
Iteration 4: Mean Squared Error: 0.02806436259062383, R^2: -0.02861713293779533
Iteration 5: Mean Squared Error: 0.13652481701752622, R^2: 0.0389868720918678
Iteration 6: Mean Squared Error: 357.40106267513875, R^2: -0.03695783476217751
Iteration 7: Mean Squared Error: 0.1021386658644816, R^2: -0.0753456850640064
Iteration 8: Mean Squared Error: 0.29864748521770806, R^2: -0.06540463315736966
Iteration 9: Mean Squared Error: 0.10806422929561789, R^2: -0.0003974643819644186
Iteration 10: Mean Squared Error: 0.328665534105124, R^2: 0.020039285888946767
Iteration 11: Mean Squared Error: 357.52593429028764, R^2: -0.03794222312586171
Iteration 12: Mean Squared Error: 357.5468279021539, R^2: -0.03820164352598687
Iteration 13: Mean Squared Error: 0.3626539620039

In [5]:
print(len(broken_models))

x_test=data_splits[0][1]  # Get the test set from the first broken model's data split
y_test=data_splits[0][3]  # Get the test labels from the first broken model's data split

25
